<a href="https://colab.research.google.com/github/DarkLord-13/Machine-Learning-01/blob/main/preprocessing_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required structure:**
_____________________________________________

main_directory

  -> sub_dir_1

  -> sub_dir_2

  -> ...

In [ ]:
from zipfile import ZipFile
import shutil
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
#shutil.rmtree('test')

Unzipping the file

In [ ]:
data_folder_name = 'train' # give your zipped file name

In [ ]:
dataset = f'/content/{data_folder_name}.zip'
with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('dataset is extracted')

dataset is extracted


Creating the mirror directory

In [ ]:
input_folder_path = data_folder_name # give your input folder path

output_folder_path = f'{input_folder_path}_aug' # give your output folder path

if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=360,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    vertical_flip=True,
    horizontal_flip=True,
    fill_mode='nearest'
)

def augment_images(input_images, num_augmentations=7):
    augmented_images = []

    # Reshape the input_images array to fit the expected shape by the ImageDataGenerator
    input_images = input_images.reshape((input_images.shape[0], input_images.shape[1], input_images.shape[2], -1))

    # Loop through each input image
    for img in input_images:
        # Expand dimensions to fit the expected shape by the ImageDataGenerator
        img = np.expand_dims(img, axis=0)

        # Flow and generate augmented images
        aug_iter = datagen.flow(img, batch_size=1)

        for _ in range(num_augmentations):
            batch = aug_iter.next()
            augmented_images.extend(batch)

    return augmented_images

Saving Images

In [ ]:
def save_images(arr, folder_path):
    for idx, img in enumerate(arr):
        file_path = f'{folder_path}/{idx}.png'
        cv2.imwrite(file_path, img)

Creating mirror subdirectory

In [ ]:
subfolders = os.listdir(input_folder_path)
for i in subfolders:
  os.makedirs(f'/content/{output_folder_path}/{i}')

Pre-Processing images and saving in their respective sub-directories

In [ ]:
def expand2square(pil_img):
  background_color = (255, 255, 255)
  pil_img = pil_img.convert('RGB')
  width, height = pil_img.size
  if width == height:
      return pil_img
  elif width > height:
      result = Image.new(pil_img.mode, (width, width), background_color)
      result.paste(pil_img, (0, (width - height) // 2))
      return result
  else:
      result = Image.new(pil_img.mode, (height, height), background_color)
      result.paste(pil_img, ((height - width) // 2, 0))
      return result

In [ ]:
for i in subfolders:
  images = os.listdir(f'/content/{input_folder_path}/{i}')
  image_list = []

  for j in images:
    fpp = f'/content/{input_folder_path}/{i}/{j}'
    img = Image.open(fpp)

    if img.mode != 'RGB':
        img = img.convert('RGB')

    img = ImageEnhance.Contrast(img)
    img = img.enhance(1.5)
    img = expand2square(img)
    img = ImageOps.expand(img, border=40, fill='white')
    img = img.resize((112, 112))
    img = np.array(img)
    image_list.append(img)

  num_augmentations = 7
  images_aug = augment_images(np.array(image_list), num_augmentations)

  save_images(images_aug, f'/content/{output_folder_path}/{i}')

Zipping the aug images folder

In [ ]:
shutil.make_archive(output_folder_path, 'zip', output_folder_path)

'/content/train_aug.zip'